In [1]:
from numpy import array, empty, linalg
import numpy as np
from numpy import copy
import pivoting as pv

### PROBLEM 1 - Resistors circuit

Alternative **a)** using the Kirchhoff's law to obtain others equations and knowing that $(V_{0} = 0)$:

$$ \frac{V_{2} - V_{0}}{R} + \frac{V_{2} - V_{4}}{R} + \frac{V_{2} - V_{1}}{R} = 0$$

$$ -V_{1} + 3V_{2} +0V_{3} - V_{4} = 0 \tag {1}$$

$$ \frac{V_{3} - V_{4}}{R} + \frac{V_{3} - V_{1}}{R} + \frac{V_{3} - V_{+}}{R} = 0$$

$$ -V_{1} + 0V_{2} +3V_{3} - V_{4} = V_{+} \tag {2}$$

$$ \frac{V_{4} - V_{0}}{R} + \frac{V_{4} - V_{1}}{R} + \frac{V_{4} - V_{3}}{R} + \frac{V_{4} - V_{2}}{R}= 0$$

$$ -V_{1} -V_{2} -V_{3} + 4V_{4} = 0 \tag{3}$$

The fourth equation $\ 4V_{1} -V_{2} -V_{3} -V_{4} = V_{+}\ $  is in pdf of the list 

Putting the indices of equations in the matriz A and knowing that $(V_{+} = 0)$:

In [2]:
A_res = array([[4, -1, -1, -1], 
           [-1, 3, 0, -1], 
           [-1, 0, 3, -1], 
           [-1, -1, -1, 4]], float)

v_res = array([5, 0, 5, 0], float)
N = len(v_res)

Alternative **b)** Solving the matriz with Gauss' elimination:

In [3]:
for m in range(N):
    div = A_res[m, m]
    A_res[m, :] /= div
    v_res[m] /= div

    for i in range(m+1, N):
        mult = A_res[i, m]
        A_res[i, :] -= mult*A_res[m, :]
        v_res[i] -= mult*v_res[m]

vs = empty(N, float)
for m in range(N-1, -1, -1):
    vs[m] = v_res[m]
    
    for i in  range(m+1, N):
        vs[m] -= A_res[m, i]*vs[i] 

print(f'Solution: {vs}')

Solution: [3.         1.66666667 3.33333333 2.        ]


Alternative **c)** Using the linalg.solve function by numpy library, we have:

In [4]:
Gausslinalg = np.linalg.solve(A_res, v_res)
print(f'Solution: {Gausslinalg}')

Solution: [3.         1.66666667 3.33333333 2.        ]


### PROBLEM 2 - Include the pivotiozation method

First we create the matriz **A** and the vector **v**:

In [5]:
A_non_pivo = np.array([[2, 1, 4, 1],
             [3, 4, -1, -1],
             [1, -4, 1, 5],
             [2, -2, 1, 3]], float)

A_pivo = np.array([[0, 1, 4, 1],
             [3, 4, -1, -1],
             [1, -4, 1, 5],
             [2, -2, 1, 3]], float)

v_pivo = array([-4, 3, 9, 7], float)
N = len(v_pivo)

Second we need indentify the **index of line** below of pivo that **your value is non-zero**, to all m columns:

After that, we exchange the **line that have pivo equal zero** with first line below this one **that has the pivo non-zero**

We make the same with respective lines from vector v

Now we can make the Gauss' elimination for that column **m**, then the same thing for the others

For this, i'll create a function to use after if i want

In [6]:
def gauss_elimination_pivoting(A, v, N):
    
    for m in range(N):
        
        # Second we need indentify the **index of line** below of pivo that **your value is non-zero**, to all m columns:
        index_pivo_non_zero = m
        while A[index_pivo_non_zero, m] == 0:
            index_pivo_non_zero += 1
    
        # After that, we exchange the **line that have pivo equal zero** with first line below this one **that has the pivo non-zero**
        for value in range(N):
            temp_A = A[m, value]
            A[m, value] = A[index_pivo_non_zero, value]
            A[index_pivo_non_zero, value] = temp_A
    
        # We make the same with respective lines from vector v
        temp_v = v[m]
        v[m] = v[index_pivo_non_zero]
        v[index_pivo_non_zero] = temp_v
    
        # Now we can make the Gauss' elimination for that column **m**, then the same thing for the others
        div = A[m, m]
        A[m, :] /= div
        v[m] /= div
    
        for i in range(m+1, N):
            mult = A[i, m]
            A[i, :] -= mult*A[m, :]
            v[i] -= mult*v[m]
        
    vs = empty(N, float)
    for m in range(N-1, -1, -1):
        vs[m] = v[m]
        
        for i in  range(m+1, N):
            vs[m] -= A[m, i]*vs[i]
    return vs

In [7]:
non_pivoting = pv.gauss_elimination_pivoting(A_non_pivo, v_pivo, N)
print(f'without pivotization: {non_pivoting}')

without pivotization: [ 2. -1. -2.  1.]


In [8]:
with_pivoting = pv.gauss_elimination_pivoting(A_pivo, v_pivo, N)
print(f'with pivotization: {with_pivoting}')

with pivotization: [ 1.78095238 -0.77142857  0.03809524 -1.38095238]


### PROBLEM 3 - LU Decomposition

Alternative **a)** First we declare the **matriz A** and the **vector v**

In [9]:
A_lu = array([[2, 1, 4, 1], 
           [3, 4, -1, -1], 
           [1, -4, 1, 5], 
           [2, -2, 1, 3]], float)

v_lu = array([-4, 3, 9, 7], float)
N = len(v_lu)

The variables **line** and **column** represent the initial line and column for each step

Outside of "for" we **divide all values by the momentary pivo ( U [ line, column ] )**  and **add the column to L** for each step

Inside of "for" we calculate that we need for others lines, after that we add 1 to line and column to the next step

In [10]:
line, column = 0, 0

U = A_lu.copy()
L = np.zeros([4, 4])

while column <= 3:
    L[line:, column] = U[line:, column]
    U[line, column:] = U[line, column:]/U[line, column]

    for i in range(line+1, N):

        U[i, column:] = U[i, column:] - U[i, column] * U[line, column:]
        if i > 0:
            U[i, column:] = U[i, column:] - U[i, column]*U[line, column:]

    line += 1
    column += 1

print(f'U:\n {U}\n')
print(f'L:\n {L}\n')

U:
 [[ 1.   0.5  2.   0.5]
 [ 0.   1.  -2.8 -1. ]
 [ 0.   0.   1.  -0. ]
 [ 0.   0.   0.   1. ]]

L:
 [[  2.    0.    0.    0. ]
 [  3.    2.5   0.    0. ]
 [  1.   -4.5 -13.6   0. ]
 [  2.   -3.  -11.4  -1. ]]



Now we can calculate the LU and **we will see that is equal to matrix A_lu**

In [11]:
LU = np.zeros([4, 4])

for i in range(N):
    for j in range(N):
        value = 0
        for k in range(N):
            value += L[i, k]*U[k, j]
            LU[i, j] = value
        
print(f'LU:\n {LU}\n')

LU:
 [[ 2.  1.  4.  1.]
 [ 3.  4. -1. -1.]
 [ 1. -4.  1.  5.]
 [ 2. -2.  1.  3.]]



In [12]:
solve = pv.gauss_elimination_pivoting(A_lu, v_lu, N)
solve

array([ 2., -1., -2.,  1.])

Alternative **b)** Now we use the bunded function by ufrn archive and prove using linalg function

In [13]:
def banded(Aa,va,up,down):

    # Copy the inputs and determine the size of the system
    A = copy(Aa)
    v = copy(va)
    N = len(v)

    # Gaussian elimination
    # for m in range(N):

    #     # Normalization factor
    #     div = A[up,m]

    #     # Update the vector first
    #     v[m] /= div
    #     for k in range(1,down+1):
    #         if m+k<N:
    #             v[m+k] -= A[up+k,m]*v[m]

        # Now normalize the pivot row of A and subtract from lower ones
    for i in range(up):
        j = m + up - i
        if j<N:
            A[i,j] /= div
            for k in range(1,down+1):
                A[i+k,j] -= A[up+k,m]*A[i,j]

    # Backsubstitution
    for m in range(N-2,-1,-1):
        for i in range(up):
            j = m + up - i
            if j<N:
                v[m] -= A[i,j]*v[j]

    return v

In [14]:
using_bunded_function = banded(A_lu,v_lu, 0, 0)
print(f'using_bunded_function: {using_bunded_function}')

using_bunded_function: [-2.   3.6 -2.   1. ]


In [15]:
using_linalg_function = np.linalg.solve(A_lu, v_lu)
print(f'using_linalg_function: {using_linalg_function}')

using_linalg_function: [ 2. -1. -2.  1.]


### PROBLEM 4 - A chain of resistors

###PROBLEM 5 - 